## METHOD 1 : Using Beautiful Soup

In [17]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [18]:
#Lets define the URL

url="https://www.worldometers.info/coronavirus/"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [19]:
#we pick the id of the table we want to scrape and extract HTML code for that particular table only
covid_table = soup.find("table", attrs={"id": "main_table_countries_today"})

In [20]:
#the head will form our columns
head = covid_table.thead.find_all("tr") 
head #the headers are contained in this HTML code

[<tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New Deaths/1M pop</th>
 <th width="30">Active Cases/1M pop</th>
 </tr>]

In [21]:
headings = []
for th in head[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    print(th.text)
    #headings.append(td.b.text.replace('\n', ' ').strip())
    headings.append(th.text.replace("\n","").strip())
print(headings)

#
Country,Other
TotalCases
NewCases
TotalDeaths
NewDeaths
TotalRecovered
NewRecovered
ActiveCases
Serious,Critical
Tot Cases/1M pop
Deaths/1M pop
TotalTests
Tests/
1M pop

Population
Continent
1 Caseevery X ppl
1 Deathevery X ppl
1 Testevery X ppl
New Cases/1M pop
New Deaths/1M pop
Active Cases/1M pop
['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']


In [22]:
body = covid_table.tbody.find_all("tr") 
body[0] #here is one example of HTML snippet for one row

<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td></td>
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>40,309,824</td>
<td>+4,915</td>
<td>911,109</td>
<td>+202</td>
<td>33,625,554</td>
<td>+2,681</td>
<td>5,773,161</td>
<td>11,461</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="North America" style="display:none;">North America</td>
<td>
</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>

In [23]:
#lets declare empty list data that will hold all rows data
data = []
for r in range(1,len(body)):
    row = [] # empty lsit to hold one row data
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
        #append row data to row after removing newlines escape and triming unnecesary spaces
    data.append(row)
    
# data contains all the rows excluding header
# row contains data for one row

In [24]:
#We can now pass data into a pandas dataframe
#with headings as the columns
df = pd.DataFrame(data,columns=headings)
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,Asia,"54,242,089","+21,014","762,222",+334,"51,540,359","+31,044","1,939,508","26,834",...,,,,Asia,,,,,,
1,,South America,"31,403,277","+2,118","964,614",+85,"28,428,687","+2,623","2,009,976","31,771",...,,,,South America,,,,,,
2,,Europe,"47,387,981",+252,"1,091,039",+5,"44,867,729","+3,678","1,429,213","9,027",...,,,,Europe,,,,,,
3,,Africa,"5,201,976",,"137,032",,"4,614,411",,"450,533","2,973",...,,,,Africa,,,,,,
4,,Oceania,"71,051",+9,"1,259",,"67,835",,"1,957",10,...,,,,Australia/Oceania,,,,,,
5,,,721,,15,,706,,0,0,...,,,,,,,,,,
6,,World,"178,616,919","+28,308","3,867,290",+626,"163,145,281","+40,026","11,604,348","82,076",...,,,,All,,,,,,
7,1,USA,"34,393,269",,"616,920",,"28,675,929",,"5,100,420","4,160",...,"498,230,075","1,496,767","332,870,823",North America,10,540,1,,,"15,323"
8,2,India,"29,823,546",+782,"385,167",,"28,678,390","+7,511","759,989","8,944",...,"389,207,637","279,392","1,393,049,733",Asia,47,"3,617",4,0.6,,546
9,3,Brazil,"17,802,176",,"498,621",,"16,136,968",,"1,166,587","8,318",...,"52,714,701","246,315","214,013,244",South America,12,429,4,,,"5,451"


In [25]:
data = df[df["#"]!=""].reset_index(drop=True)
# Data points with # value are the countries of the world while the data points with
# null values for # columns are features like continents totals etc
data = data.drop_duplicates(subset = ["Country,Other"])
#Reason to drop duplicates : Worldometer reports data for 3 days: today and 2 days back
#I found out that removing duplicates removes the values for the bast two days and keep today's

In [26]:
data.columns

Index(['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests',
       'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl',
       '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop',
       'New Deaths/1M pop', 'Active Cases/1M pop'],
      dtype='object')

In [27]:
# Columns to keep
cols = ['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'TotalTests',"Continent"]

In [28]:
data_final = data[cols]
data_final.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
0,USA,"34,393,269",,"616,920",,"28,675,929",,"5,100,420","4,160","498,230,075",North America
1,India,"29,823,546",+782,"385,167",,"28,678,390","+7,511","759,989","8,944","389,207,637",Asia
2,Brazil,"17,802,176",,"498,621",,"16,136,968",,"1,166,587","8,318","52,714,701",South America
3,France,"5,752,872",,"110,702",,"5,546,870",,"95,300","1,740","90,215,804",Europe
4,Turkey,"5,359,728",,"49,071",,"5,224,224",,"86,433",856,"58,123,990",Asia


In [29]:
#Once you are happy then you can save the dataframe as csv or xlsx
#data_final.to_csv("covid.csv",index=False)

In [32]:
#Here is a list of continents
list(data_final["Continent"].unique())

['North America',
 'Asia',
 'South America',
 'Europe',
 'Africa',
 'Australia/Oceania',
 '']

In [36]:
#Then you can filter by continent
data_final[data_final["Continent"]=="Africa"].head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
18,South Africa,"1,796,589",,"58,441",,"1,632,182",,"105,966",546,"12,383,955",Africa
42,Morocco,"525,924",,"9,233",,"512,937",,"3,754",101,"6,690,836",Africa
53,Tunisia,"378,982",,"13,874",,"330,331",,"34,777",424,"1,597,918",Africa
65,Egypt,"276,190",,"15,791",,"204,701",,"55,698",90,"2,869,589",Africa
66,Ethiopia,"274,899",,"4,276",,"253,634",,"16,989",228,"2,813,953",Africa


## METHOD 2 : USING XPath

In [37]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [38]:
#Rememeber that rows on HTML are stored between <tr>..</tr> tags
url = "https://www.worldometers.info/coronavirus/"
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr') #note that we are only using tr because we have only one table on the site
#if we had many then we could have needed to have the XPath so that we are specific.
#XPath can be accessed by inspecting elements of a site. 
print(len(list(tr_elements))) #number of all rows including the header row

717


In [39]:
#Lets deal with the header row first
col=[]
i=0#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"#"
2:"Country,Other"
3:"TotalCases"
4:"NewCases"
5:"TotalDeaths"
6:"NewDeaths"
7:"TotalRecovered"
8:"NewRecovered"
9:"ActiveCases"
10:"Serious,Critical"
11:"Tot Cases/1M pop"
12:"Deaths/1M pop"
13:"TotalTests"
14:"Tests/
1M pop
"
15:"Population"
16:"Continent"
17:"1 Caseevery X ppl"
18:"1 Deathevery X ppl"
19:"1 Testevery X ppl"
20:"New Cases/1M pop"
21:"New Deaths/1M pop"
22:"Active Cases/1M pop"


In [44]:
# Since out first row is the header, data is stored on the second row onwards
for j in range(1, len(tr_elements)):
    # T is our j'th row
    T = tr_elements[j]

    # If row is not of size 19, the //tr data is not from our table
#     if len(T) != 19:
#         break

    # i is the index of our column
    i = 0

    # Iterate through each element of the row
    for t in T.iterchildren():
        data = t.text_content()
        # Check if row is empty
        if i > 0:
            # Convert any numerical value to integers
            try:
                data = int(data)
            except:
                pass
        # Append the data to the empty list of the i'th column
        col[i][1].append(data)
        # Increment i for the next column
        i += 1

#ideally all columns must have the same number of rows
print([len(C) for (title,C) in col]) #for this case we have 695

[716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716, 716]


In [45]:
#From here the same explanation holds like in method 1 above
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
data = df[df["#"]!=""].reset_index(drop=True)
data = data.drop_duplicates(subset = ["Country,Other"])

In [46]:
# Columns to keep
cols = ['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'TotalTests',"Continent"]

data_final = data[cols]
#Assume we want to pick only these 5 countries
c = ["South Africa" , "Rwanda","Senegal","Ghana","Cameroon"]
data = data_final[data_final["Country,Other"].isin(c)].reset_index(drop=True)
data.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
0,South Africa,"1,796,589",,"58,441",,"1,632,182",,"105,966",546,"12,383,955",Africa
1,Ghana,"94,913",,793,,"92,881",,"1,239",9,"1,230,125",Africa
2,Cameroon,"80,328",,"1,313",,"78,162",,853,152,"1,718,937",Africa
3,Senegal,"42,259",,"1,158",,"40,767",,334,6,"545,258",Africa
4,Rwanda,"30,048",,378,,"26,393",,"3,277",16,"1,535,396",Africa
